In [33]:
import pandas as pd
import numpy as np
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting
from pypfopt import EfficientFrontier, objective_functions

def litterman(train_returns):
    weights = {}
    prior = {}
    returns_bl = {}
    for tick in train_returns.keys():
        # Среднее по прошлому перформансу для каждого актива
        prior[tick] = np.mean(train_returns.get(tick))
    priors = pd.Series(prior)

    for tick in train_returns.keys():
        returns_bl[tick] = pd.Series(train_returns.get(tick), name = tick)

    concatter = {}
    for tick in train_returns.keys():
        # Использование последних 25 тиков для оценки ковариационной матрицы
        concatter[tick] = list(returns_bl.get(tick))

    allresults = pd.DataFrame(concatter)
    # ЛеДуа-Вульф-оценка
    S = risk_models.CovarianceShrinkage(allresults, returns_data=True).ledoit_wolf()
    viewdict = {}
    for tick in train_returns.keys():
        # Среднее по последним 25 тикам
        viewdict[tick] = train_returns.get(tick).mean()

    intervals = []
    for tick in train_returns.keys():
        intervals.append((np.percentile(train_returns.get(tick), 25), np.percentile(train_returns.get(tick), 75)))

    variances = []
    for lb, ub in intervals:
        sigma = (ub - lb)/2
        variances.append(sigma ** 2)
    omega = np.diag(variances)
    bl = BlackLittermanModel(S, absolute_views=viewdict, omega=omega, pi = priors)
    ret_bl = bl.bl_returns()
    S_bl = bl.bl_cov()
    ef = EfficientFrontier(ret_bl, S_bl, weight_bounds=(-3,3))
    try:
        ef.max_quadratic_utility()
        weights = ef.clean_weights()
    except:
        print('except')
        pass
    return weights

In [34]:
litterman(dt)

OrderedDict([('BZ=F', 0.07557),
             ('GC=F', 0.29388),
             ('NG=F', 0.05306),
             ('SI=F', 0.20354),
             ('^GSPC', 0.37395)])

In [31]:
import yfinance  as yf

data = yf.download(tickers='GC=F SI=F ^GSPC BZ=F NG=F', period='6y', interval='1mo')['Close'].iloc[:-4].dropna()
data[['^GSPC', 'BZ=F', 'NG=F']] = -data[['^GSPC', 'BZ=F', 'NG=F']].pct_change()
data[['GC=F', 'SI=F']] = data[['GC=F', 'SI=F']].pct_change()
data = data.iloc[1:]
dt = {}
for col in data.columns:
    dt[col] = data[col].values

[*********************100%***********************]  5 of 5 completed


In [43]:
weights = litterman(dt)

returns = np.zeros(len(data))
for key in list(weights.keys()):
    returns += weights.get(key) * data[col].values

In [54]:
np.quantile(returns, 0.03, interpolation='higher')

-0.07006466690171775